# GALAXY CLASSIFIER

This IPython Notebook contains the Python code for implementing a Convolution Neural Network(CNN) Architecture for Classification of Galaxy Images into it's 3 corresponding classes- Spiral type, Ireegular type and Elliptical type. The model was trained on Nvidia 960MX GPU followed by an intensive training on the NVIDIA DGX 1 Octa Tesla V100 Supercomputer servers using technologies like Putty and WinSCP. On training for 40 epochs, it was observed the training accuracy was at 95.00% with training loss at 15.37% while Validation accuracy was at 94.75% and Validation loss at 15.31%. The Training set containing 3 classes were a total of 3232 images while the Validation set containing the same number of classes contained 1190 images. 

In [1]:
import keras
import tensorflow
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation
import h5py
import warnings
from keras import backend as K
import os
from keras.layers import LeakyReLU
import timeit
warnings.filterwarnings('ignore')

C:\Users\Akshat Aman\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
num_classes=3
input_shape=(256,256,3)

In [3]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(3, 3), input_shape=input_shape))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(32, (3, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(32))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 16)      448       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 254, 254, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 252, 252, 32)      4640      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 252, 252, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 126, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 124, 124, 32)      9248      
_________________________________________________________________
lea

In [27]:
start = timeit.default_timer()
model.compile(loss=keras.losses.categorical_crossentropy, optimizer = keras.optimizers.Adam(), metrics=['accuracy'])
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'training_set',
        target_size=(256, 256),
        batch_size=64,
        class_mode='categorical' )

test_set = test_datagen.flow_from_directory(
        'val_set',
        target_size=(256, 256),
        batch_size=64,
        class_mode='categorical')


classifier = model.fit_generator(
        training_set,
        steps_per_epoch=10,
        epochs=200,
        validation_data=test_set,
        validation_steps=100)

end = timeit.default_timer()
print("Time Taken to run the model:",end - start, "seconds") 

Found 3232 images belonging to 3 classes.
Found 1190 images belonging to 3 classes.
Epoch 1/40
10/10 [==============================] - 145s 15s/step - loss: 0.8451 - acc: 0.6109 - val_loss: 0.6192 - val_acc: 0.7006
Epoch 2/40
10/10 [==============================] - 147s 15s/step - loss: 0.5722 - acc: 0.7203 - val_loss: 0.4876 - val_acc: 0.8201
Epoch 3/40
10/10 [==============================] - 148s 15s/step - loss: 0.4967 - acc: 0.7797 - val_loss: 0.4246 - val_acc: 0.8818
Epoch 4/40
10/10 [==============================] - 145s 15s/step - loss: 0.4141 - acc: 0.8375 - val_loss: 0.3179 - val_acc: 0.8837
Epoch 5/40
10/10 [==============================] - 146s 15s/step - loss: 0.3800 - acc: 0.8688 - val_loss: 0.3024 - val_acc: 0.9010
Epoch 6/40
10/10 [==============================] - 150s 15s/step - loss: 0.3018 - acc: 0.9016 - val_loss: 0.3526 - val_acc: 0.8831
Epoch 7/40
10/10 [==============================] - 157s 16s/step - loss: 0.3167 - acc: 0.8922 - val_loss: 0.2720 - val_acc:

In [29]:
model.save_weights('model.h5')